In [ ]:
from airflow.decorators import dag
from pendulum import datetime
from airflow.providers.apache.kafka.sensors.kafka import (AwaitMessageTriggerFunctionSensor,)

from airflow.operators.trigger_dagrum import TriggerDagRunOperator
import json
import uuid

pet_moods=['sleepy','angry']
KAFKA_TOPIC="my_topic"

def listen_function(message, pet_moods=[]):

  message_content=json.loads(message.value())
  print(f"full meaaage:{message_content}")
  pet_ame=message_content["pet_name"]
  pet_mood_post_treat=message_content["pet_mood_post_treat"]
  final_treat=message_content["final_treat"]

  if final_treat:
    if pet_mood_post in pet_moods:
      return pet_name, pet_mood_post




def event_triggered_function(message, **context):

  pet_name= message[0]
  pet_mood_post=message[1]
  print(f"due t {pet_name in {pet_mood_post}, awlk is initinated}")

  TriggerDagRunOperator(

                        trigger_dag_id="walK_pet",
                        task_id=f"ttoggered _downstream{uuid.uuid4()}",
                        wait_for_comple=True,
                        conf={
                            "pet_name":pet_name
                        },
                        poke_interval=20,




  ).execute(context)

  print(f"the walk has conlcude dand {pet_name} is now happi;y taking a nap")



@dag(

     start_date=datetime(2023,4,1),
     schedule="@continuos",
     max_active_runs=1,
     catchup=False,
     render_template_as_native_obj=True,


)



def listen_to_the_stream():


  listen_for_mood=AwaitMessageTriggerFunctionSensor(
    task_id="lsiten_for_mood",
    kafka_config_id="kafa_listener",
    topics=[KAFKA_TOPIC],

    apply_function="listen_to_the_stream.listen_function",
    poll_interval=5,
    poll_timeout=1,
    apply_function_kwargs={"pet_moods":PET_MOODS}
    event_triggered_function=event_trigered_function,


    )




=========================================================================


from airflow.decorators import dag
from pendulum import datetime
from airflow.providers.apache.kafka.sensors.kafka import AwaitMessageTriggerFunctionSensor
from airflow.operators.trigger_dagrun import TriggerDagRunOperator
import json
import uuid


KAFKA_TOPIC = "inventory_updates"

# Threshold for low inventory
LOW_INVENTORY_THRESHOLD = 20  # Example threshold

# Function to process inventory messages
def process_inventory(message, low_inventory_threshold):
    message_content = json.loads(message.value())
    item_name = message_content["item_name"]
    current_quantity = message_content["quantity"]

    if current_quantity < low_inventory_threshold:
        return item_name, current_quantity

# Function to handle triggering workflow for low inventory
def trigger_inventory_workflow(message, **context):
    item_name = message[0]
    current_quantity = message[1]
    print(f"Low inventory detected for {item_name}. Triggering workflow.")

    TriggerDagRunOperator(
        trigger_dag_id="order_processing_workflow",
        task_id=f"trigger_order_processing_{uuid.uuid4()}",
        conf={
            "item_name": item_name,
            "current_quantity": current_quantity
        },
        poke_interval=60,
    ).execute(context)
    print(f"Order processing workflow triggered for {item_name}.")

# Define the main DAG
@dag(
    start_date=datetime(2023, 4, 1),
    schedule_interval="@hourly",
    max_active_runs=1,
    catchup=False,
    render_template_as_native_obj=True,
)
def monitor_inventory():
    # Sensor to listen for inventory updates
    listen_for_inventory = AwaitMessageTriggerFunctionSensor(
        task_id="listen_for_inventory",
        kafka_config_id="kafka_listener_config",
        topics=[KAFKA_TOPIC],
        apply_function="monitor_inventory.process_inventory",
        poll_interval=30,
        poll_timeout=10,
        apply_function_kwargs={"low_inventory_threshold": LOW_INVENTORY_THRESHOLD},
        event_triggered_function="monitor_inventory.trigger_inventory_workflow",
    )

# Instantiate the DAG
inventory_monitor_dag = monitor_inventory()


